# Chapitre 4   


Dans ce TP, nous allons effectuer le finetuning d'un modèle transformer pré-entraîné (BERT) sur une des tâches du benchmark GLUE.  

Le benchmark [GLUE](https://gluebenchmark.com/) (General Language Understanding Evaluation) est un ensemble de dataset pour l'apprentissage et l'évaluation des modèles NLP. Il contient plusieurs "tâches" avec un dataset train sur lequel on peut entraîner le modèle et un dataset test pour l'évaluer. Cette diversité de tâches permet d'apprécier la capacité de compréhension du langage du modèle.  

Nous utiliserons la library datasets de huggingface qui contient les datasets pour chacune des tâches du benchmark : [glue](https://huggingface.co/datasets/glue). La description des tâches est spécifiée dans la page du dataset.

Vous êtes libre de choisir la tâche sur laquelle vous souhaitez travailler. Par défaut, ce notebook suppose que la tâche choisie est la tâche MRPC (Microsoft Research Paraphrase Corpus) qui consiste à prédire si des paires de phrases sont sémantiquement équivalentes.  

Pour ce TP, nous utiliserons la library transformers et torch pour effectuer l'entraînement du modèle.  

Chargez le dataset Glue pour la tâche MRPC

In [326]:
!pip install transformers datasets torch
from datasets import load_dataset

# Charger le dataset MRPC
dataset = load_dataset("glue", "mrpc")
print(dataset)

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


DatasetDict({
    train: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 3668
    })
    validation: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 408
    })
    test: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 1725
    })
})


Chargez le modèle pré-entraîné BERT (bert-base-uncased) ainsi que son tokenizer.

Utilisez la classe AutoModelForSequenceClassification ou BertModelForSequenceClassification.

In [327]:
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification

# Charger le tokenizer pour le modèle bert-base-uncased
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
# Charger le modèle BERT pré-entraîné pour une tâche de classification
model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)
# Texte exemple
sentence1 = "The cat sat on the mat."
sentence2 = "The feline was resting on the rug."

# Tokenisation
inputs = tokenizer(sentence1, sentence2, return_tensors="pt", padding=True, truncation=True)

# Passage dans le modèle
outputs = model(**inputs)

# Résultat des logits
print(outputs.logits)

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tensor([[0.0564, 0.4043]], grad_fn=<AddmmBackward0>)


Affichez quelques informations de base sur le modèle et son tokenizer pour. Exemples:  
- Nombre de paramètres  
- Architecture du modèle  
- Taille du vocabulaire  
- ...

Affichez quelques exemples du dataset

In [328]:
print("Nombre de paramètres :", model.num_parameters())
print("Architecture du modèle :", model.config.architectures)
print("Nombre de couches :", model.config.num_hidden_layers)
print("Taille des embeddings :", model.config.hidden_size)
print("Taille du vocabulaire :", tokenizer.vocab_size)
print("Token spécial [CLS] :", tokenizer.cls_token)
print("Token spécial [SEP] :", tokenizer.sep_token)
print("Token spécial [PAD] :", tokenizer.pad_token)

for i in range(3):
    print(f"Exemple {i + 1}:")
    print("Phrase 1 :", dataset['train'][i]['sentence1'])
    print("Phrase 2 :", dataset['train'][i]['sentence2'])
    print("Label :", dataset['train'][i]['label'])

Nombre de paramètres : 109483778
Architecture du modèle : ['BertForMaskedLM']
Nombre de couches : 12
Taille des embeddings : 768
Taille du vocabulaire : 30522
Token spécial [CLS] : [CLS]
Token spécial [SEP] : [SEP]
Token spécial [PAD] : [PAD]
Exemple 1:
Phrase 1 : Amrozi accused his brother , whom he called " the witness " , of deliberately distorting his evidence .
Phrase 2 : Referring to him as only " the witness " , Amrozi accused his brother of deliberately distorting his evidence .
Label : 1
Exemple 2:
Phrase 1 : Yucaipa owned Dominick 's before selling the chain to Safeway in 1998 for $ 2.5 billion .
Phrase 2 : Yucaipa bought Dominick 's in 1995 for $ 693 million and sold it to Safeway for $ 1.8 billion in 1998 .
Label : 0
Exemple 3:
Phrase 1 : They had published an advertisement on the Internet on June 10 , offering the cargo for sale , he added .
Phrase 2 : On June 10 , the ship 's owners had published an advertisement on the Internet , offering the explosives for sale .
Label 

Tokenisez quelques exemples du dataset et interpretez l'output de la fonction de tokenisation

In [329]:
from transformers import AutoTokenizer
from datasets import load_dataset

# Tokeniser quelques exemples
for i in range(3):
    example = dataset['train'][i]
    tokens = tokenizer(example['sentence1'], example['sentence2'], truncation=True, padding=True, return_tensors="pt",max_length=65)
    
    # Affichage des informations
    print(f"Exemple {i + 1}:")
    print("Phrase 1 :", example['sentence1'])
    print("Phrase 2 :", example['sentence2'])
    print("Tokens :", tokens['input_ids'])
    print("Attention Mask :", tokens['attention_mask'])
    print("Token Type IDs :", tokens['token_type_ids'])
    print("-" * 50)


Exemple 1:
Phrase 1 : Amrozi accused his brother , whom he called " the witness " , of deliberately distorting his evidence .
Phrase 2 : Referring to him as only " the witness " , Amrozi accused his brother of deliberately distorting his evidence .
Tokens : tensor([[  101,  2572,  3217,  5831,  5496,  2010,  2567,  1010,  3183,  2002,
          2170,  1000,  1996,  7409,  1000,  1010,  1997,  9969,  4487, 23809,
          3436,  2010,  3350,  1012,   102,  7727,  2000,  2032,  2004,  2069,
          1000,  1996,  7409,  1000,  1010,  2572,  3217,  5831,  5496,  2010,
          2567,  1997,  9969,  4487, 23809,  3436,  2010,  3350,  1012,   102]])
Attention Mask : tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1]])
Token Type IDs : tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

Utilisez la fonction decode du tokenizer afin d'avoir une représentation plus lisible du résultat de tokenisation. Vous remarquerez l'ajout des tokens [CLS], [SEP] notamment.

In [330]:
# Tokeniser quelques exemples du dataset
tokenized_example_1 = tokenizer(
    "Amrozi accused his brother, whom he called 'the witness', of deliberately distorting his evidence.",
    padding='max_length', truncation=True, return_tensors='pt'
)
tokenized_example_2 = tokenizer(
    "Referring to him as only 'the witness', Amrozi accused his brother of deliberately distorting his evidence.",
    padding='max_length', truncation=True, return_tensors='pt'
)

# Utilisation de la fonction decode pour afficher une représentation lisible
decoded_example_1 = tokenizer.decode(
    tokenized_example_1["input_ids"][0], skip_special_tokens=False
)
decoded_example_2 = tokenizer.decode(
    tokenized_example_2["input_ids"][0], skip_special_tokens=False
)

print("Phrase tokenisée et décodée - Exemple 1 :")
print(decoded_example_1)
print("\nPhrase tokenisée et décodée - Exemple 2 :")
print(decoded_example_2)


Phrase tokenisée et décodée - Exemple 1 :
[CLS] amrozi accused his brother, whom he called ' the witness ', of deliberately distorting his evidence. [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]

Tokenisez tout le dataset.

Conseil : Utilisez la fonction map de la library datasets.

In [331]:
# Tokenisation de tout le dataset avec la fonction map
def tokenize_function(examples):
    return tokenizer(examples['sentence1'], examples['sentence2'], truncation=True, padding=True, return_tensors="pt",max_length=65)

# Application de la fonction de tokenisation
tokenized_dataset = dataset.map(tokenize_function, batched=True)

# Affichage des clés du dataset tokenisé pour confirmation
print(tokenized_dataset)


DatasetDict({
    train: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 3668
    })
    validation: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 408
    })
    test: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 1725
    })
})


Une fois le dataset tokenisé, il faut créer un DataLoader qui va donner les exemples au modèle durant l'entraînement.   
Définissez un train_dataloader et un eval_dataloader.  

Pour le train_dataloader, spécifiez `shuffle=True` afin que les exemples soient donnés dans un ordre aléatoire au modèle.

Il est à noter que le modèle s'attend à prendre en entrée des exemples sous forme de **dictionnaires** de **tenseurs torch** et ayant les clés suivantes uniquement:  
- attention_mask  
- input_ids  
- labels  
- token_type_ids  

Vous aurez probablement à définir une fonction de collate à donner en paramètre à vos dataloaders dans le paramètre `collate_fn`.  Cette fonction est appelée par le dataloader juste avant de donner l'exemple au modèle afin de lui donner le bon format (mettre uniquement les clés attendues par le modèle, padder les exemples à une longueur fixe, ...).  

- [How to use collate_fn](https://discuss.pytorch.org/t/how-to-use-collate-fn/27181)  
- [How to use 'collate_fn' with dataloaders?](https://stackoverflow.com/questions/65279115/how-to-use-collate-fn-with-dataloaders)  

In [332]:
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence
import torch

# 1. Dataset personnalisé
class CustomDataset(Dataset):
    def __init__(self, tokenized_data):
        self.tokenized_data = tokenized_data  # Votre dataset tokenisé

    def __len__(self):
        return len(self.tokenized_data)

    def __getitem__(self, idx):
        return self.tokenized_data[idx]
        
# Séparer le dataset en train et évaluation
train_dataset = tokenized_dataset['train']  # Données d'entraînement
eval_dataset = tokenized_dataset['validation']  # Données de validation

# 2. Fonction collate_fn

import torch
from torch.nn.utils.rnn import pad_sequence

def collate_fn(batch):
    # Définir la longueur maximale pour le padding
    max_len = 65  # La longueur maximale que vous souhaitez pour les séquences
    
    input_ids = [torch.tensor(item['input_ids'][:max_len]) for item in batch]
    attention_masks = [torch.tensor(item['attention_mask'][:max_len]) for item in batch]
    token_type_ids = [torch.tensor(item['token_type_ids'][:max_len]) for item in batch]
    
    labels = [torch.tensor(item['label']) for item in batch]

    # Padding à la longueur maximale
    input_ids = pad_sequence(input_ids, batch_first=True, padding_value=0)
    attention_masks = pad_sequence(attention_masks, batch_first=True, padding_value=0)
    token_type_ids = pad_sequence(token_type_ids, batch_first=True, padding_value=0)
    
    batch_dict = {
        'input_ids': input_ids,
        'attention_mask': attention_masks,
        'labels': labels,
        'token_type_ids': token_type_ids

    }
    
    if labels is not None:
        batch_dict['labels'] = torch.stack(labels)

    return batch_dict




# 3. Charger votre dataset tokenisé (supposons que vous avez `train_dataset` et `eval_dataset` déjà définis)
train_dataset = CustomDataset(train_dataset)  # Votre dataset tokenisé pour l'entraînement
eval_dataset = CustomDataset(eval_dataset)    # Votre dataset tokenisé pour l'évaluation

# 4. Créer les DataLoader
train_dataloader = DataLoader(
    train_dataset,
    batch_size=8,
    shuffle=True,
    collate_fn=collate_fn
)

eval_dataloader = DataLoader(
    eval_dataset,
    batch_size=8,
    shuffle=False,
    collate_fn=collate_fn
)


Afin de vérifier que votre Dataloader fonctionne bien, vous pouvez itérer dessus comme ci-dessous :

In [333]:
for batch in train_dataloader:
    break
{k: v.shape for k, v in batch.items()}

{'input_ids': torch.Size([8, 65]),
 'attention_mask': torch.Size([8, 65]),
 'labels': torch.Size([8]),
 'token_type_ids': torch.Size([8, 65])}

Le code ne devrait pas crasher et vous devriez avoir une output similaire à:  
```
{
  'attention_mask': torch.Size([8, 65]),
  'input_ids': torch.Size([8, 65]),
  'labels': torch.Size([8]),
  'token_type_ids': torch.Size([8, 65])
}
```

Vous pouvez vous assurer que le modèle prend en entrée les données du dataloader en lui donnant en entrée le batch précédent. Voici un exemple de code:  

In [334]:
outputs = model(**batch)
print(outputs.loss, outputs.logits.shape)

tensor(0.4949, grad_fn=<NllLossBackward0>) torch.Size([8, 2])


Maintenant que tout est en place, nous allons lancer l'entraînement.  
Définissez l'optimizer à utiliser durant l'apprentissage. Par exemple AdamW.  
Définissez aussi les paramètres d'apprentissage, par exemple:  
- Nombre d'epochs  
- Nombre de training steps  
- Un schedule pour le learning rate ou un learning rate fixe  

In [335]:
from torch.optim import AdamW
from transformers import get_linear_schedule_with_warmup

# Définir l'optimiseur
optimizer = AdamW(model.parameters(), lr=2e-5)
# Nombre d'époques et de training steps
num_epochs = 3
num_training_steps = len(train_dataloader) * num_epochs


# Définir un scheduler de learning rate avec warm-up
lr_scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=0,  # Nombre d'étapes pour le warm-up (souvent entre 0 et 10% des étapes totales)
    num_training_steps=num_training_steps
)



Vous pouvez aussi utiliser le Trainer de la library transformers.  

In [336]:
from transformers import Trainer, TrainingArguments
from sklearn.metrics import accuracy_score

# Définir la fonction de calcul des métriques
def compute_metrics(p):
    preds = p.predictions.argmax(axis=1)  # Prédictions en prenant l'index avec la probabilité la plus élevée
    labels = p.label_ids
    return {"eval_accuracy": accuracy_score(labels, preds)}  # Renvoie eval_accuracy comme clé

# Définir les arguments d'entraînement
training_args = TrainingArguments(
    output_dir='./results',             # Répertoire pour sauvegarder les résultats
    num_train_epochs=3,                 # Nombre d'époques
    per_device_train_batch_size=8,      # Taille du batch pour l'entraînement
    per_device_eval_batch_size=8,       # Taille du batch pour l'évaluation
    warmup_steps=0,                     # Nombre de warm-up steps
    weight_decay=0.01,                  # Poids pour la régularisation L2
    logging_dir='./logs',               # Répertoire des logs
    logging_steps=10,                   # Fréquence de logging
    evaluation_strategy="epoch",        # Évaluation après chaque époque
    save_strategy="epoch",              # Sauvegarde après chaque époque
    load_best_model_at_end=True,        # Charger le meilleur modèle à la fin
    metric_for_best_model="eval_accuracy",  # Critère pour sélectionner le meilleur modèle
    report_to="none"                   # Pas de rapport vers W&B
)

# Créer un objet Trainer
trainer = Trainer(
    model=model,                        # Modèle à entraîner
    args=training_args,                 # Les arguments d'entraînement
    train_dataset=train_dataset,         # Dataset d'entraînement
    eval_dataset=eval_dataset,           # Dataset d'évaluation
    tokenizer=tokenizer,                 # Tokenizer
    compute_metrics=compute_metrics,     # Fonction pour calculer les métriques
    optimizers=(optimizer, lr_scheduler) # Optimiseur et scheduler
)

# Lancer l'entraînement
trainer.train()



/opt/conda/lib/python3.10/site-packages/transformers/training_args.py:1545: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy
1,0.561500,0.368206,0.843137
2,0.238100,0.400536,0.860294
3,0.026400,0.518481,0.867647


TrainOutput(global_step=1377, training_loss=0.32814860387218975, metrics={'train_runtime': 128.8769, 'train_samples_per_second': 85.384, 'train_steps_per_second': 10.685, 'total_flos': 367564088782800.0, 'train_loss': 0.32814860387218975, 'epoch': 3.0})

Ecrivez la boucle d'évaluation de votre modèle. Pour cela, vous pouvez utiliser la library evaluate de huggingface qui facilite le calcul des métriques. Elle contient la métrique spécifique à la tâche qui nous intéresse. Voici un exemple de comment la charger :  

Voici la page associée pour avoir de la doc sur les métriques GLUE : [Metric: glue](https://huggingface.co/spaces/evaluate-metric/glue)

In [341]:
import torch
import evaluate
from tqdm import tqdm

# Charger la métrique (par exemple, accuracy)
accuracy_metric = evaluate.load("accuracy")

# Définir le périphérique (GPU ou CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Fonction d'évaluation
def evaluate_model(model, dataloader, device):
    model.eval()  # Mettre le modèle en mode évaluation
    all_preds = []
    all_labels = []
    
    # Déplacer le modèle vers le bon périphérique
    model.to(device)
    
    # Désactivation de la mise à jour des gradients
    with torch.no_grad():
        for batch in tqdm(dataloader, desc="Evaluating"):
            # Déplacer les données vers le bon périphérique (GPU ou CPU)
            batch = {k: v.to(device) for k, v in batch.items()}
            
            # Effectuer une passe avant (forward pass)
            outputs = model(**batch)
            logits = outputs.logits
            
            # Convertir les logits en prédictions de classes
            preds = torch.argmax(logits, dim=-1)
            
            # Ajouter les prédictions et labels à leurs listes respectives
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(batch['labels'].cpu().numpy())
    
    # Calcul de la métrique (ex. : accuracy)
    results = accuracy_metric.compute(predictions=all_preds, references=all_labels)
    
    return results

# Appel de la fonction d'évaluation sur votre modèle
results = evaluate_model(model, eval_dataloader, device)
print("Evaluation Results:", results)


Evaluating: 100%|██████████| 51/51 [00:01<00:00, 45.31it/s]

Evaluation Results: {'accuracy': 0.8676470588235294}


# [Optionnel] Génération de texte avec GPT2

Si vous souhaitez expérimenter avec un language model est faire de la génération de texte. Vous pouvez utiliser le modèle GPT 2 qui est un language model causal. Il permet donc d'effectuer de la génération de texte contrairement à BERT qui est un modèle bidirectionnel et qui n'est pas fait pour cela.  

Vous pouvez essayer différents paramètres de génération et algorithme de décodage en vous basant sur le tutoriel/guide suivant : [How to generate text: using different decoding methods for language generation with Transformers](https://huggingface.co/blog/how-to-generate).